<a href="https://colab.research.google.com/github/myDSMLProjects/Deep-Learning/blob/master/KFold_optimization_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

df = pd.concat([df, pd.get_dummies(df['job'], prefix='job')], axis=1)
df.drop('job', axis=1, inplace=True)
df = pd.concat([df, pd.get_dummies(df['area'], prefix='area')], axis=1)
df.drop('area', axis=1, inplace=True)
df = pd.concat([df, pd.get_dummies(df['product'], prefix='product')], axis=1)
df.drop('product', axis=1, inplace=True)

med = df['income'].median()
df['income'] = df['income'].fillna(med)

df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['subscriptions'] = zscore(df['subscriptions'])

x_columns = df.columns.drop('age').drop('id')
x = df[x_columns].values
y = df['age'].values

In [3]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

#cross validate
kf = KFold(n_splits=5, shuffle=True, random_state=42)

oos_y = []
oos_pred = []

fold = 0
for train, test in kf.split(x):
  fold += 1
  print(f'Fold: {fold}')

  x_train = x[train]
  x_test = x[test]
  y_train = y[train]
  y_test = y[test]

  model = Sequential()
  model.add(Dense(25, input_dim = x.shape[1], activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train , y_train, validation_data=(x_test, y_test), verbose=0, epochs=500)

  pred = model.predict(x_test)
  oos_y.append(y_test)
  oos_pred.append(pred)

  print(f'Fold score (RMSE): {np.sqrt(metrics.mean_squared_error(y_test, pred))}')

oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(f'Final out of sample score RMSE : {np.sqrt(metrics.mean_squared_error(oos_y, oos_pred))}')

Fold: 1
Fold score (RMSE): 0.6518271120053309
Fold: 2
Fold score (RMSE): 0.5275155939276147
Fold: 3
Fold score (RMSE): 0.5892802189275216
Fold: 4
Fold score (RMSE): 0.4374763315667425
Fold: 5
Fold score (RMSE): 1.0007784488290543
Final out of sample score RMSE : 0.6698276653771572
